In [1]:
import numpy as np
import geopy as gp
import pandas as pd
from geopy.geocoders import Nominatim
from pathlib import Path
from tqdm import tqdm
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

2024-12-20 09:15:17.505 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\where-am-i


In [ ]:
from google.colab import drive
import os
import shutil

# Define paths
DRIVE_FOLDER = '/content/drive/MyDrive/Colab/where-am-i/data'  # Folder where chunks are stored
EXTRACTION_FOLDER = '/content/train'  # Folder to extract images

os.makedirs(EXTRACTION_FOLDER, exist_ok=True)

# List chunks
chunks = [f for f in os.listdir(DRIVE_FOLDER) if f == 'train_15.zip']

# Extract all chunks
for chunk in chunks:
    chunk_path = os.path.join(DRIVE_FOLDER, chunk)
    print(f"Extracting {chunk_path}...")
    shutil.unpack_archive(chunk_path, EXTRACTION_FOLDER)

print("All images extracted successfully!")


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab/where-am-i/train.csv')
INTERIM_DATA_DIR = '/content/'

ids = set(df_train.loc[:, 'id'].values.tolist())
dic_ids = []
for root, dirs, files in os.walk('/content/train'):
    for file in tqdm(files):
        id = int(file.split('.jpg')[0])
        if id in ids:
            dic_ids.append(id)
df_train = df_train.set_index(keys='id').loc[dic_ids,]
trainset = df_train.reset_index().iloc[:int(len(df_train) * 0.9)]
valset = df_train.reset_index().iloc[int(len(df_train) * 0.9):]

In [19]:
#df_train.drop(columns=['coarse', 'medium', 'fine']).to_csv(INTERIM_DATA_DIR / 'train/train.csv', index = False)
df_train = pd.read_csv(INTERIM_DATA_DIR / 'train.csv')
df_train = pd.merge(df_train, pd.read_parquet(INTERIM_DATA_DIR / 'hashed_annos.parquet').loc[:, ['id', 'latitude', 'longitude']], on='id')#.drop(columns=['coarse_i','medium_i','fine_i'])


In [17]:
#df_train.to_csv(INTERIM_DATA_DIR / 'pos_train.csv', index=False)
df_train = pd.read_csv(INTERIM_DATA_DIR / 'pos_train.csv')
trainset = df_train.iloc[:int(len(df_train) * 0.9)]
valset = df_train.iloc[int(len(df_train) * 0.9):]

In [21]:
import os
import pandas as pd
from torchvision.io import decode_image, read_file
from torch.utils.data import Dataset
import torch
from pathlib import Path

class OSVImageDataset(Dataset):
    def __init__(self, annotations_df, img_dir, transform=None):
        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        #todo: idx using image id?
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0]) + '.jpg')
        image = decode_image(img_path).float() / 255.0
        label = torch.tensor([self.img_labels.iloc[idx, 1], self.img_labels.iloc[idx, 2]])
        if self.transform:
            image = self.transform(image)
        image = image.clamp(0, 1)
        return image, label

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from torchvision import datasets, transforms
#from src.base.OSVImageDataset import OSVImageDataset
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor
from torchvision.transforms import v2

BATCH_SIZE = 64
KERNEL_SIZE = 16 #16x16 patch
CHANNELS = 3 #rgb
RESIZE = 224
EMBED_DIM = CHANNELS * KERNEL_SIZE ** 2
NUM_PATCHES = ((RESIZE + 0 - KERNEL_SIZE)//KERNEL_SIZE + 1) ** 2
MODEL_NAME = 'google/vit-base-patch16-224-in21k'

#Using values the ViT was trained on
processor = ViTImageProcessor.from_pretrained(MODEL_NAME, do_rescale = False, return_tensors = 'pt')

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = v2.Normalize(mean=image_mean, std=image_std)

train_transform = v2.Compose([
      v2.Resize((processor.size["height"], processor.size["width"])),
      #v2.RandomHorizontalFlip(0.4),
      #v2.RandomVerticalFlip(0.1),
      #v2.RandomApply(transforms=[v2.RandomRotation(degrees=(0, 90))], p=0.5),
      #v2.RandomApply(transforms=[v2.ColorJitter(brightness=.3, hue=.1)], p=0.3),
      #v2.RandomApply(transforms=[v2.GaussianBlur(kernel_size=(5, 9))], p=0.3),
      normalize
 ])

test_transform = v2.Compose([
    v2.Resize((processor.size["height"], processor.size["width"])),
    normalize
])

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
print(f"Using device: {device}")

train_dataset = OSVImageDataset(annotations_df = trainset, img_dir=INTERIM_DATA_DIR / 'train', transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = OSVImageDataset(annotations_df = valset, img_dir = INTERIM_DATA_DIR / 'train', transform=test_transform)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


Using device: cuda


In [15]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image

class GeoLocator(nn.Module):
    def __init__(self):
        super(GeoLocator, self).__init__()
        
        self.backbone = ViTModel.from_pretrained(MODEL_NAME)
        
        self.layer1 = nn.Linear(self.backbone.config.hidden_size, self.backbone.config.hidden_size)
        self.norm1 = nn.LayerNorm(self.backbone.config.hidden_size)
        self.dropout1 = nn.Dropout(p=0.05)  
        
        self.layer2 = nn.Linear(self.backbone.config.hidden_size, 512)  # 512: embedding size of location encoder
        self.norm2 = nn.LayerNorm(512)
        self.dropout2 = nn.Dropout(p=0.05)
        
        self.layer3 = nn.Linear(512, 512)

    def forward(self, x):
        # Extract last hidden state from the ViT backbone
        x1 = self.backbone(x).last_hidden_state
        x1 = x1[:, 0, :]  # Use CLS token only

        # First layer with normalization and dropout
        a1 = func.leaky_relu(self.norm1(self.layer1(x1)))
        a1 = self.dropout1(a1)
        
        # Second layer with normalization and dropout
        a2 = func.leaky_relu(self.norm2(self.layer2(a1)))
        a2 = self.dropout2(a2)
        
        # Output layer
        output = self.layer3(a2)
        
        return output

In [16]:
import gc
from torch.amp import autocast, GradScaler
from loguru import logger

model = GeoLocator().to(device=device)
#freezing backbone
for param in model.backbone.parameters():
    param.requires_grad = False

#optimizer for custom layers only
optimizer = torch.optim.AdamW([
    {'params': model.layer1.parameters()},
    {'params': model.layer2.parameters()},
    {'params': model.layer3.parameters()}
], lr = 0.001)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
criterion = nn.CosineEmbeddingLoss()


In [ ]:
from geoclip import LocationEncoder

gps_encoder = LocationEncoder().to(device=device)
scaler = GradScaler()
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, labels in train_dataloader:
        images = images.to(device=device)
        labels = gps_encoder(labels.float().to(device=device))
        with autocast(device_type=device.__str__()):
            output = model(images)
            ones = torch.ones(BATCH_SIZE).to(device=device)
            loss = criterion(output, labels, ones)

        #clearing memory so that my gpu doesn't die :)
        del images, labels, ones
        gc.collect()

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    if (epoch + 1) % 1 == 0:
        logger.info(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

        #clearing memory so that my gpu doesn't die :)
        del loss
        gc.collect
        torch.cuda.empty_cache()

In [21]:
#from src.config import MODELS_DIR

nn_path = "geonn_t.pt"
torch.save(model.state_dict(), str(nn_path))

In [ ]:
model.eval()
with torch.no_grad():
    for images, labels in tqdm(val_dataloader):
        images = images.to(device=device)
        labels = labels.to(device=device)
        coarse_output, _, fine_output = model(images)

        coarse_true = np.concatenate((coarse_true, labels[:,0].cpu()), axis=0)
        coarse_pred = np.concatenate((coarse_pred, coarse_output.cpu()), axis=0)
        fine_true = np.concatenate((fine_true, labels[:, 2].cpu()), axis=0)
        fine_pred = np.concatenate((fine_pred, fine_output.cpu()), axis=0)
        
        del images, labels, coarse_output, fine_output
        gc.collect
        torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import top_k_accuracy_score

print(f'Top K Accuracy Fine: {top_k_accuracy_score(fine_true, fine_pred, k=5, labels=[i for i in range(FINE)]) * 100}')
print(f'Top K Accuracy Output: {top_k_accuracy_score(coarse_true, coarse_pred, k=5, labels=[i for i in range(COARSE)]) * 100}')

In [ ]:
((1.2e6/64) * 4)/3600

In [ ]:
((1.2e6/64) * 2)/3600

In [ ]:
((1.2e6/1024) * 6)/3600

In [ ]:
((3.2e4/64) * 2)/60

## Single Batch Perf check

In [18]:
import time
import torch
from geoclip import LocationEncoder

gps_encoder = LocationEncoder().to(device=device)
scaler = GradScaler()
num_epochs = 10

start = time.perf_counter()
c1 = 0
model.train()
temp = 0
for images, labels in train_dataloader:
    images = images.to(device=device)
    labels = gps_encoder(labels.float().to(device=device))
    c1 = time.perf_counter()
    with autocast(device_type=device.__str__()):
        output = model(images)
        ones = torch.ones(BATCH_SIZE).to(device=device)
        loss = criterion(output, labels, ones)

    #clearing memory so that my gpu doesn't die :)
    del images, labels, ones
    gc.collect()

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    
    del loss
    gc.collect()

    torch.cuda.empty_cache()
    if temp == 0:
      break
end = time.perf_counter()
print(f'total time for {BATCH_SIZE}: {end - start}')
print(f'transform time for {BATCH_SIZE}: {c1 - start}')

c:\Users\Harshavardhan Patil\.virtualenvs\where-am-i-s7vJJwrF\lib\site-packages\geoclip\model\location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load

total time for 64: 0.8369551999494433
transform time for 64: 0.8253547999775037


In [ ]:
import time
import torch
from geoclip import LocationEncoder

gps_encoder = LocationEncoder().to(device=device)
scaler = GradScaler()
num_epochs = 10

start = time.perf_counter()
c1 = 0
model.train()
temp = 0
for images, labels in train_dataloader:
    images = images.to(device=device)
    labels = gps_encoder(labels.float().to(device=device))
    c1 = time.perf_counter()
    with autocast(device_type=device.__str__()):
        output = model(images)
        ones = torch.ones(BATCH_SIZE).to(device=device)
        loss = criterion(output, labels, ones)

    #clearing memory so that my gpu doesn't die :)
    del images, labels, ones
    gc.collect()

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    
    del loss
    gc.collect()

    torch.cuda.empty_cache()
    if temp == 0:
      break
end = time.perf_counter()
print(f'total time for {BATCH_SIZE}: {end - start}')
print(f'transform time for {BATCH_SIZE}: {c1 - start}')

In [ ]:
df_train = df_train.rename(columns={'coarse_i':'coarse', 'medium_i':'medium', 'fine_i':'fine'})

In [31]:
df = df_train.iloc[:1000]

In [30]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_top_k_similar_embeddings(query_embedding, candidate_embeddings, k):
    """
    Find the top k embeddings from the candidates based on cosine similarity.
    
    Args:
        query_embedding (torch.Tensor): The embedding to compare against, shape (1, 512).
        candidate_embeddings (torch.Tensor): Candidate embeddings, shape (N, 512).
        k (int): Number of top embeddings to return.

    Returns:
        top_k_indices (list): Indices of the top k most similar embeddings.
    """
    similarities = cosine_similarity(query_embedding.cpu().numpy(), candidate_embeddings.cpu().numpy())[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]  # Sort in descending order
    return top_k_indices

def predict_locations(images, labels, dataloader, GeoLocator, gps_encoder, dataframe, k=5):
    """
    Predict top k latitude and longitude/geohashes for given images.

    Args:
        images (torch.Tensor): Batch of input images.
        labels (torch.Tensor): Ground truth latitudes and longitudes.
        dataloader (DataLoader): Dataloader for images and labels.
        GeoLocator (torch.nn.Module): Trained model for predicting location embeddings.
        gps_encoder (function): Function that encodes (lat, lon) into a 512D vector.
        dataframe (pd.DataFrame): Dataframe with columns ['id', 'latitude', 'longitude', 'coarse', 'medium', 'fine'].
        k (int): Number of top similar locations to return.

    Returns:
        results (list): A list of dictionaries with final top k predictions.
    """
    results = []

    # Precompute embeddings for unique coarse, medium, and fine geohashes
    unique_coarse = dataframe.drop_duplicates(subset=['coarse'])
    unique_medium = dataframe.drop_duplicates(subset=['medium'])
    unique_fine = dataframe.drop_duplicates(subset=['fine'])

    coarse_coords = torch.tensor(unique_coarse[['latitude', 'longitude']].values, dtype=torch.float32)
    medium_coords = torch.tensor(unique_medium[['latitude', 'longitude']].values, dtype=torch.float32)
    fine_coords = torch.tensor(unique_fine[['latitude', 'longitude']].values, dtype=torch.float32)

    unique_coarse['embedding'] = gps_encoder(coarse_coords)
    unique_medium['embedding'] = gps_encoder(medium_coords)
    unique_fine['embedding'] = gps_encoder(fine_coords)

    coarse_embeddings = torch.stack(unique_coarse['embedding'].tolist())
    medium_embeddings = torch.stack(unique_medium['embedding'].tolist())
    fine_embeddings = torch.stack(unique_fine['embedding'].tolist())

    for batch_images, batch_labels in dataloader:
        batch_results = []

        # Predict embeddings using GeoLocator
        predicted_embeddings = GeoLocator(batch_images)

        for pred_embedding in predicted_embeddings:
            # Step 1: Find top k coarse geohashes
            top_k_coarse_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), coarse_embeddings, k)
            top_k_coarse_candidates = unique_coarse.iloc[top_k_coarse_indices]

            # Step 2: Find top k medium geohashes
            medium_subset = unique_medium[unique_medium['coarse'].isin(top_k_coarse_candidates['coarse'])]
            medium_embeddings_subset = torch.stack(medium_subset['embedding'].tolist())
            top_k_medium_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), medium_embeddings_subset, k)
            top_k_medium_candidates = medium_subset.iloc[top_k_medium_indices]

            # Step 3: Find top k fine geohashes
            fine_subset = unique_fine[unique_fine['medium'].isin(top_k_medium_candidates['medium'])]
            fine_embeddings_subset = torch.stack(fine_subset['embedding'].tolist())
            top_k_fine_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), fine_embeddings_subset, k)
            final_top_k = fine_subset.iloc[top_k_fine_indices]

            # Retrieve final top k latitudes and longitudes
            batch_results.append(final_top_k[['latitude', 'longitude', 'fine']])

        results.extend(batch_results)

    return results


## Embedding to GPS Decoder

In [18]:
df = df_train.drop(columns=['id', 'coarse_i', 'medium_i', 'fine_i'])

In [23]:
from geoclip import LocationEncoder
import gc

gps_encoder = LocationEncoder().to(device=device)

coords = torch.tensor(df[["latitude", "longitude"]].values, dtype=torch.float32).to(device)

# Generate embeddings
with torch.no_grad():
    embeddings = gps_encoder(coords).cpu().numpy()

embedding_df = pd.DataFrame(
    embeddings, 
    columns=[f"embedding_{i+1}" for i in range(embeddings.shape[1])]
)

# Concatenate the original DataFrame with the embeddings DataFrame
df = pd.concat([df, embedding_df], axis=1)

c:\Users\Harshavardhan Patil\.virtualenvs\where-am-i-s7vJJwrF\lib\site-packages\geoclip\model\location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load

In [2]:
df = pd.read_parquet(INTERIM_DATA_DIR / 'embeddings.parquet').drop(columns=['id'])

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmbeddingToGPSDecoder(nn.Module):
    def __init__(self):
        super(EmbeddingToGPSDecoder, self).__init__()

        # First layer: 512 (embedding size) -> 512
        self.layer1 = nn.Linear(512, 512)
        self.norm1 = nn.LayerNorm(512)

        # Second layer: 512 -> 256
        self.layer2 = nn.Linear(512, 256)
        self.norm2 = nn.LayerNorm(256)

        # Output layer: 256 -> 2 (latitude and longitude)
        self.output_layer = nn.Linear(256, 2)

    def forward(self, x):
        # First layer with normalization and dropout
        x1 = F.leaky_relu(self.norm1(self.layer1(x)))

        # Second layer with normalization and dropout
        x2 = F.leaky_relu(self.norm2(self.layer2(x1)))

        # Output layer for predicting latitude and longitude
        gps_coordinates = self.output_layer(x2)

        return gps_coordinates


In [25]:
import os
import pandas as pd
from torchvision.io import decode_image, read_file
from torch.utils.data import Dataset
import torch
from pathlib import Path

class OSVEmbeddingsDataset(Dataset):
    def __init__(self, annotations_df):
        self.df = annotations_df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.df.iloc[idx, 2:], dtype=torch.float)
        label = torch.tensor([self.df.iloc[idx, 0], self.df.iloc[idx, 1]])
        return embedding, label

In [26]:
import torch
import torch.nn as nn

class GeodesicDistanceLoss(nn.Module):
    def __init__(self, radius=6371):
        super(GeodesicDistanceLoss, self).__init__()
        self.radius = radius  # Earth's radius in kilometers

    def forward(self, pred, target):
        # Convert latitude and longitude from degrees to radians
        pred_rad = torch.deg2rad(pred)
        target_rad = torch.deg2rad(target)

        # Split latitudes and longitudes
        lat1, lon1 = pred_rad[:, 0], pred_rad[:, 1]
        lat2, lon2 = target_rad[:, 0], target_rad[:, 1]

        # Haversine formula
        delta_lat = lat2 - lat1
        delta_lon = lon2 - lon1
        a = torch.sin(delta_lat / 2) ** 2 + \
            torch.cos(lat1) * torch.cos(lat2) * torch.sin(delta_lon / 2) ** 2
        c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

        # Distance in kilometers
        distance = self.radius * c
        return distance.mean()


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from torchvision import datasets, transforms
#from src.base.OSVImageDataset import OSVImageDataset
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor
from torchvision.transforms import v2

BATCH_SIZE = 1024

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
print(f"Using device: {device}")

train_dataset = OSVEmbeddingsDataset(annotations_df = df)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


Using device: cuda


In [28]:
from geoclip import LocationEncoder
import gc
from torch.amp import autocast, GradScaler
from loguru import logger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from src.config import MODELS_DIR

model = EmbeddingToGPSDecoder()
model.load_state_dict(torch.load(MODELS_DIR / "reversenn.pt", weights_only=True))
model = model.to(device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
criterion = GeodesicDistanceLoss()

num_epochs = 5
for epoch in (range(num_epochs)):
    model.train()
    for embeddings, labels in tqdm(train_dataloader):
        embeddings = embeddings.to(device=device)
        labels = labels.to(device=device)
        
        output = model(embeddings)
        loss = criterion(output, labels)

        #clearing memory so that my gpu doesn't die :)
        del output, embeddings, labels
        gc.collect()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step(loss) 
    if (epoch + 1) % 1 == 0:
        logger.info(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

        del loss
        gc.collect
        torch.cuda.empty_cache()

  0%|          | 0/1178 [00:00<?, ?it/s]C:\Users\Harshavardhan Patil\AppData\Local\Temp\ipykernel_19844\4128849953.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  embedding = torch.tensor(self.df.iloc[idx, 2:], dtype=torch.float)
100%|██████████| 1178/1178 [14:52<00:00,  1.32it/s]


2024-12-20 09:55:58.043 | INFO     | __main__:<module>:36 - Epoch [1/5], Loss: 26.3345


  1%|          | 9/1178 [00:07<15:13,  1.28it/s]


KeyboardInterrupt: 

In [112]:
from src.config import MODELS_DIR

nn_path = "reversenn.pt"
torch.save(model.state_dict(), MODELS_DIR / nn_path)

In [121]:
#df.to_parquet(INTERIM_DATA_DIR / 'embeddings.parquet')

In [119]:
df_train.loc[:,['id','latitude','longitude']].equals(pd.concat([df_train['id'], df], axis=1).loc[:, ['id','latitude','longitude']])

True

In [120]:
df = pd.concat([df_train['id'], df], axis=1)

In [14]:
from geoclip import LocationEncoder
import gc
from torch.amp import autocast, GradScaler
from loguru import logger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from src.config import MODELS_DIR

model = EmbeddingToGPSDecoder()
model.load_state_dict(torch.load(MODELS_DIR / "reversenn.pt", weights_only=True))
model.eval()
gps_encoder = LocationEncoder()

embeds = gps_encoder(torch.tensor([[41.659354389516, -111.86601253359]]))
output = model(embeds)

In [15]:
output

tensor([[  41.3965, -111.4439]], grad_fn=<AddmmBackward0>)

In [16]:
from geopy import distance

og = (41.659354389516, -111.86601253359)
decoded = (41.3965, -111.4439)

print(distance.distance(og, decoded).km)


45.75356838199693
